# CaBI Data Downloading and Simple Cleaning


# Overview

The first seciton uses a chrome browser to download all the zip files from CaBI site to a local directory. Then unpacks all the zip files to the same directory. Within these files is the trip history by bike, including starting and ending station, duration of the trip, member type, date/time of trip.

The second section downloads a json page of all the station locations and their region ID. Then is downloads another json page that maps the region IDs to the DMV regions as defined by CaBi.

The third section preprocess the raw csv files
    -extracts the day of the week, hour, and day for each ride
    -megres the region codes into the trip data. 
    -converts duration to minutes from miliseconds
    -stacks the qarterly CSVs into years




In [2]:
### download_all_cabi.py

from splinter import Browser
import os
import urllib.request
import shutil
import zipfile


def get_links():
    ### You'll need to find a chrome browser or firefox browser to launch this function
    browser = Browser('chrome')
    browser.visit('https://s3.amazonaws.com/capitalbikeshare-data/index.html')

    links = browser.find_by_css('#tbody-content a')
    #links = [link for link in links if link['href']]
    zip_links = []
    for link in links:
        link_text = link['href']
        with urllib.request.urlopen(link_text) as response:
            subtype = response.info().get_content_subtype()
            if subtype == 'zip':
                zip_links.append(link_text)

    return zip_links


def download_links(links):
    download_dir = '../cabi_data'
    if not os.path.exists(download_dir):
        os.mkdir(download_dir)
    for link in links:
        file_name = os.path.join(download_dir, link.split('/')[-1])
        print(file_name)
        # save file to disk
        with urllib.request.urlopen(link) as response, open(file_name, 'wb') as out_file:
            shutil.copyfileobj(response, out_file)
        with zipfile.ZipFile(file_name, "r") as zip_ref:
            zip_ref.extractall(download_dir)

links = get_links()
download_links(links)


../cabi_data\2010_trip_history_data.zip
../cabi_data\2011_trip_history_data.zip
../cabi_data\2012_trip_history_data.zip
../cabi_data\2013_trip_history_data.zip
../cabi_data\2014_trip_history_data.zip
../cabi_data\2015_trip_history_data.zip
../cabi_data\2016_trip_history_data.zip
../cabi_data\2017_trip_history_data.zip


In [9]:
####CaBi_Stations


import pandas as pd
import requests
import numpy as np
import time
import os

# Load station information
station_url = "https://gbfs.capitalbikeshare.com/gbfs/en/station_information.json"
stations = requests.get(station_url).json()
station_df = pd.DataFrame(stations['data']['stations'])
station_df = station_df[['lat', 'lon', 'region_id', 'short_name']].copy()
# Default any missing region ids to DC (only example as of 2/21 is new station at Anacostia Park)
station_df['region_id'] = np.where(
    station_df['region_id'].isnull(), 42, station_df['region_id'])

# Convert region_id to str from float
station_df['region_id'] = station_df['region_id'].astype(int).astype(str)

# Load region information
region_url = "https://gbfs.capitalbikeshare.com/gbfs/en/system_regions.json"
regions = requests.get(region_url).json()
regions_df = pd.DataFrame(regions['data']['regions'])

# Merge region information onto stations
station_df = station_df.merge(
    regions_df, left_on='region_id', right_on='region_id', how='left')
station_df.rename(index=str, columns={'name': 'region_name'}, inplace=True)
print(len(station_df))
print(station_df['region_name'].value_counts())

# Define Abbreviations for each region

region_code = {'Washington, DC': 'WDC',
               'Arlington, VA': 'ARL',
               'Montgomery County, MD (South)': 'MCS',
               'Montgomery County, MD (North)': 'MCN',
               'Alexandria, VA': 'ALX',
               'Fairfax, VA': 'FFX'}

region_code_series = pd.Series(region_code, name='region_code')
region_code_series.index.name = 'region_name'
region_code_df = region_code_series.reset_index()
station_df = station_df.merge(
    region_code_df, left_on='region_name', right_on='region_name', how='left')
station_df.drop(['region_name', 'region_id'], inplace=True, axis=1)

# Output DataFrame as CSV with timestamp
TIMESTR = time.strftime("%Y%m%d_%H%M%S")
filename = "CABI_Station_Info_" + TIMESTR + ".csv"
filepath = os.path.join("C:/Users/Noah/", filename)
station_df.to_csv(filepath, index=False)


490
Washington, DC                   269
Arlington, VA                     92
Montgomery County, MD (South)     45
Alexandria, VA                    31
Montgomery County, MD (North)     28
Fairfax, VA                       25
Name: region_name, dtype: int64


In [59]:
## Compile Bikeshare Data ##

import pandas as pd
import time
import os
import sys
import re
from datetime import date
import calendar


TIMESTR = time.strftime("%Y%m%d_%H%M%S")

# Load Cabi Trip Data as Data Frame

Final_Daily_Rides = pd.DataFrame()

all_header_files = glob2.glob('C:/Users/Noah/cabi_data//*.csv')
FILE_LIST = []
FILE_LIST_all = []
counter = 0
for file in all_header_files:
    counter = counter +1 
    FILE = re.search('([2][0][0-9][0-9][-][a-z][0-4])',file)
    FILE_LIST.append(FILE.group(0))
    FILE_LIST_all.append(FILE.group(0))
    trip_df = pd.read_csv(file)
    trip_df['source'] = FILE
    # Derive Trip Date from Start Date time stamp
    start_datetime = pd.to_datetime(trip_df['Start date'])
    end_datetime = pd.to_datetime(trip_df['End date'])
    trip_df['start_date'] = start_datetime.map(lambda x: x.date())
    trip_df['end_date'] = end_datetime.map(lambda x: x.date())
    #0-23, Military Time - EST
    trip_df['start_hour'] = start_datetime.dt.hour
    trip_df['end_hour'] = end_datetime.dt.hour
    #0-6, 0 is Monday - 6 is Sunday
    trip_df['Weekday'] = trip_df['start_date'].apply(lambda x: x.weekday())
    # Append Region Code onto Trip Data
    station_info_df = pd.read_csv('C:/Users/Noah/CABI_Station_Info_20180307_080242.csv')
    start_station_info_df = station_info_df.add_prefix('start_')
    trip_df = trip_df.merge(start_station_info_df, left_on='Start station number',
                            right_on='start_short_name', how='left')
    end_station_info_df = station_info_df.add_prefix('end_')
    trip_df = trip_df.merge(end_station_info_df, left_on='End station number',
                            right_on='end_short_name', how='left')
    trip_df['region_start_end'] = trip_df['start_region_code'] + \
        '_to_' + trip_df['end_region_code']
    trip_df['station_start_end'] = trip_df['Start station'] + \
        '_to_' + trip_df['End station']
    # Milisecond per Minute 0.0000166667 
    trip_df['Minutes'] = 0.0000166667 * trip_df['Duration (ms)']
    '''There are a handful of trips that will be dropped
    because they are either warehouse trips or the station
    no longer exists,option to find location by name instead of
    station information'''
    # print(trip_df[(trip_df['region_start_end'].isnull()) & (
    # ~trip_df['station_start_end'].str.contains('6035 Warehouse'))]['station_start_end'].value_counts())
    
    #Append to Final Dataframe
    Final_Daily_Rides = Final_Daily_Rides.append(trip_df)
    if counter == 4:
        final = '_'.join(str(v) for v in FILE_LIST)
        filename = "CABI_Daily_Trips_" + TIMESTR + "_" + str(final) + ".csv"
        filepath = os.path.join("C:/Users/Noah/Bikeshare-DC", filename)
        Final_Daily_Rides.to_csv(filepath, index=False)
        FILE_LIST =[]
        Final_Daily_Rides =pd.DataFrame()
        counter = 0
    elif file == all_header_files[0]:
        final = '_'.join(str(v) for v in FILE_LIST)
        filename = "CABI_Daily_Trips_" + TIMESTR + "_" + str(final) + ".csv"
        filepath = os.path.join("C:/Users/Noah/Bikeshare-DC", filename)
        Final_Daily_Rides.to_csv(filepath, index=False)
        FILE_LIST =[]
        Final_Daily_Rides =pd.DataFrame()
        counter = 0
    elif len(FILE_LIST_all) == len(all_header_files):
        final = '_'.join(str(v) for v in FILE_LIST)
        filename = "CABI_Daily_Trips_" + TIMESTR + "_" + str(final) + ".csv"
        filepath = os.path.join("C:/Users/Noah/Bikeshare-DC", filename)
        Final_Daily_Rides.to_csv(filepath, index=False)
    
    print(file)
    
    
#Weird Error for 2016-q3 File
#C:\Users\Noah\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
#interactivity=interactivity, compiler=compiler, result=result)
#C:/Users/Noah/cabi_data\2016-q3_trip_history_data.csv






C:/Users/Noah/cabi_data\2010-q4_trip_history_data.csv
C:/Users/Noah/cabi_data\2011-q1_trip_history_data.csv
C:/Users/Noah/cabi_data\2011-q2_trip_history_data.csv
C:/Users/Noah/cabi_data\2011-q3_trip_history_data.csv
C:/Users/Noah/cabi_data\2011-q4_trip_history_data.csv
C:/Users/Noah/cabi_data\2012-q1_trip_history_data.csv
C:/Users/Noah/cabi_data\2012-q2_trip_history_data.csv
C:/Users/Noah/cabi_data\2012-q3_trip_history_data.csv
C:/Users/Noah/cabi_data\2012-q4_trip_history_data.csv
C:/Users/Noah/cabi_data\2013-q1_trip_history_data.csv
C:/Users/Noah/cabi_data\2013-q2_trip_history_data.csv
C:/Users/Noah/cabi_data\2013-q3_trip_history_data.csv
C:/Users/Noah/cabi_data\2013-q4_trip_history_data.csv
C:/Users/Noah/cabi_data\2014-q1_trip_history_data.csv
C:/Users/Noah/cabi_data\2014-q2_trip_history_data.csv
C:/Users/Noah/cabi_data\2014-q3_trip_history_data.csv
C:/Users/Noah/cabi_data\2014-q4_trip_history_data.csv
C:/Users/Noah/cabi_data\2015-q1_trip_history_data.csv
C:/Users/Noah/cabi_data\2015

C:\Users\Noah\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


C:/Users/Noah/cabi_data\2016-q3_trip_history_data.csv
C:/Users/Noah/cabi_data\2016-q4_trip_history_data.csv
C:/Users/Noah/cabi_data\2017-q1_trip_history_data.csv
C:/Users/Noah/cabi_data\2017-q2_trip_history_data.csv
C:/Users/Noah/cabi_data\2017-q3_trip_history_data.csv
C:/Users/Noah/cabi_data\2017-q4_trip_history_data.csv
